<a href="https://colab.research.google.com/github/takahiromiura/class-data-analysis-II-2024/blob/main/%E6%BC%94%E7%BF%92(%E5%A3%B2%E4%B8%8A%E3%83%A9%E3%83%B3%E3%82%AD%E3%83%B3%E3%82%B0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 演習 (松源購買データ)

株式会社松源のデータを使って分析を行います。
データの概要は以下の通りです。

- 2024 年 10 月の日別購買データ
- 2 つの店舗のデータ (00013, 00017)
- お菓子部門 (03) のデータ

`pd.options.display.max_rows` については後で説明します。

In [1]:
import pandas as pd
# pd.options.display.max_rows = 10000

In [2]:
data = pd.read_csv("/content/snack_data.csv")
data.head()

,年月日,年月,店舗コード,部門コード,商品コード（旧JAN）,商品コード（新JAN）,商品名漢字（全角）,売上数量（全体）,売上売価（全体）
0,2023/10/01,2023/10,13,3,45044182,45044182,クロレッツＸＰシャープミント,1,96
1,2023/10/01,2023/10,13,3,45126420,45126420,キシリトールライムミント粒,1,108
2,2023/10/01,2023/10,13,3,4514062291741,4514062291741,レインボーラムネミニ,1,108
3,2023/10/01,2023/10,13,3,45141447,45141447,ドラえもんフーセンガム,2,118
4,2023/10/01,2023/10,13,3,45145957,45145957,ボンタンアメ,2,256


In [3]:
data.dtypes

,0
年月日,object
年月,object
店舗コード,int64
部門コード,int64
商品コード（旧JAN）,int64
商品コード（新JAN）,int64
商品名漢字（全角）,object
売上数量（全体）,int64
売上売価（全体）,int64


どういった商品があるのかを確認します。
`Series` の `unique` メソッドを使うと、重複なしのデータが返ってきます。
ただし、このままだと少し見づらいので、`Series` に変換します。

実行すると一部が省略されます。
デフォルトでは pandas の行数が一定以上を超えると省略されるため、この設定を変えるには最初のコードセルにある `pd.options.display.max_rows = 10000` のようにして、設定を書き換えます。
一度設定を変更すると、セッション内で常に有効になります。

In [4]:
products = pd.Series(data['商品名漢字（全角）'].unique())
products

,0
0,クロレッツＸＰシャープミント
1,キシリトールライムミント粒
2,レインボーラムネミニ
3,ドラえもんフーセンガム
4,ボンタンアメ
...,...
1865,仮面ライダーグミグレープ味 ×
1866,イタジャガワンピースｗｉｔｈプラマイド ×
1867,おつまみ通の鮭とば
1868,ブーストバイツ


商品数を確認すると、1870 個あることがわかりました。

In [5]:
len(products)

1870

試しに、全体の売上ランキングを作成してみます。
商品名で集計を行い、日毎の売上数を合計します。


In [6]:
data.groupby('商品名漢字（全角）')["売上数量（全体）"].sum().head()


,売上数量（全体）
商品名漢字（全角）,
あげころ餅 ×,9
あたりめチップ ×,6
あっさりショコラ ×,16
あっさりショコラいちご ×,35
あつまれアンパンマンＰ７８,1


見やすいように、売上数が高い順にソートします。
また、`to_frame` メソッドを使い、`Series` を `DataFrame` に変換します。

これによって、売上ランクの列を挿入できます。

In [7]:
total_sales = data.groupby('商品名漢字（全角）')["売上数量（全体）"].sum().sort_values(ascending=False).to_frame()
total_sales.head()

,売上数量（全体）
商品名漢字（全角）,
ポテトチップスうすしお,1217
厚焼きピーナッツ煎餅,882
蒲焼さん太郎,716
ポテトチップスコンソメパンチ,618
アルファベットチョコレート,536


`Series` の `rank` メソッドを使うと、値に応じたランキングをつけてくれます。
デフォルトでは小さい値から 1, 2 と続くため、売上数が多い順にするには `ascending = False` を指定します。



In [8]:
total_sales["ランキング"] = total_sales["売上数量（全体）"].rank(ascending=False)
total_sales.head()

,売上数量（全体）,ランキング
商品名漢字（全角）,,
ポテトチップスうすしお,1217,1.0
厚焼きピーナッツ煎餅,882,2.0
蒲焼さん太郎,716,3.0
ポテトチップスコンソメパンチ,618,4.0
アルファベットチョコレート,536,5.0


店舗毎に集計したい場合、`groupby` に集計する列を追加します。
複数の場合はリストで指定することに注意してください。

In [9]:
store_sales = data.groupby(["商品名漢字（全角）", "店舗コード"])["売上数量（全体）"].sum().sort_values(ascending=False)
store_sales.head()

商品名漢字（全角）    店舗コード
ポテトチップスうすしお  17       705
             13       512
蒲焼さん太郎       17       505
厚焼きピーナッツ煎餅   17       479
             13       403
Name: 売上数量（全体）, dtype: int64

これだけでも、例えば店舗によって商品の仕入れが異なるとか、売上の差が「ポテトチップスうすしお」のほうが「厚焼きピーナッツ煎餅」よりも大きいなど、様々なことがわかります。

こうしたデータ分析を踏まえて、仮説などを考えていきます。